In [ ]:
import cartopy.crs as ccrs
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import cmaps
import cartopy.feature as cfeature
import sacpy as scp
from matplotlib.lines import Line2D
import matplotlib.path as mpath  
import matplotlib as mpl
from cartopy.util import add_cyclic_point
import sys
sys.path.append('/public/home/songqh/Documents/python_packages/')
from sqh_toolbox import plot_figure
from waf import tnwaf
import warnings
warnings.filterwarnings("ignore") 
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
file = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_precipitation_1940-2024.nc")
precip_index  = file['tp'].loc[file.time.dt.year.isin(np.arange(1979,2025)) & file.time.dt.month.isin([6,7,8]),28:10,70:100].groupby('time.year').mean('time').mean(('latitude','longitude'))
precip_index  = (precip_index  - np.mean(precip_index )) / np.std(precip_index )

In [ ]:
years = np.arange(1979,2025,1)
highyear = years[np.where(precip_index>1)]
lowyear = years[np.where(precip_index<-1)]
print(highyear)
print(lowyear)

In [ ]:
ds_sst = xr.open_dataset("/public/home/songqh/my_data/SST/HadISST_sst.nc")
nino34 = ds_sst['sst'].loc[ds_sst.time.dt.year.isin(np.arange(1979,2025)) & ds_sst.time.dt.month.isin([6,7,8]),5:-5,-170:-120].mean(('longitude','latitude')).values.reshape((46,3)).mean(1)
nino34 = (nino34 - nino34.mean())/np.std(nino34)
dmi_w = ds_sst['sst'].loc[ds_sst.time.dt.year.isin(np.arange(1979,2025)) & ds_sst.time.dt.month.isin([6,7,8]),10:-10,50:70].mean(('longitude','latitude')).values.reshape((46,3)).mean(1)
dmi_e = ds_sst['sst'].loc[ds_sst.time.dt.year.isin(np.arange(1979,2025)) & ds_sst.time.dt.month.isin([6,7,8]),0:-10,90:110].mean(('longitude','latitude')).values.reshape((46,3)).mean(1)
dmi = dmi_w - dmi_e
dmi = (dmi - dmi.mean())/np.std(dmi)

In [ ]:
fig = plt.figure(figsize=(9, 4))
ax = fig.add_subplot(111)
ax.set_ylim(-2.4, 2.4)
ax.set_xlim(1978, 2025)
ax.set_detail()
ax.plot(years, precip_index, c='k', lw=2,  zorder=3)
ax.axhline(-1, lw=1.5, linestyle='--', c='g')
ax.axhline(1, lw=1.5, linestyle='--', c='g')

idx_pos = np.where(nino34 > 1.5)[0]
idx_neg = np.where(nino34 < -1.5)[0]
idx_dmi_pos = np.where(dmi > 1.5)[0]
idx_dmi_neg = np.where(dmi < -1.5)[0]

ax.scatter(years[idx_pos], [2.3]*len(idx_pos), marker='v', c='red', zorder=4, label='Nino > 1.5')
ax.scatter(years[idx_neg], [-2.3]*len(idx_neg), marker='^', c='blue', zorder=4, label='Nino < -1.5')
ax.scatter(years[idx_dmi_pos], [2.1]*len(idx_dmi_pos), marker='s', c='red', zorder=4, label='DMI > 1.5')
ax.scatter(years[idx_dmi_neg], [-2.1]*len(idx_dmi_neg), marker='s', c='blue', zorder=4, label='DMI < -1.5')

marked_years = np.unique(np.concatenate([
    years[idx_pos], years[idx_neg],
    years[idx_dmi_pos], years[idx_dmi_neg]]))

for y in highyear:
    if y not in marked_years:
        ax.axvline(y, color='r', linewidth=4, alpha=0.35)
for y in lowyear:
    if y not in marked_years:
        ax.axvline(y, color='gray', linewidth=4, alpha=0.35)

legend_elements = [
    Line2D([0], [0], color='k', lw=2, label='Precip index'),
    Line2D([0], [0], marker='^', color='w', label='El Niño',
           markerfacecolor='red', markersize=10),
    Line2D([0], [0], marker='v', color='w', label='La Niña',
           markerfacecolor='blue', markersize=10),
    Line2D([0], [0], marker='s', color='w', label='Positive IOD',
           markerfacecolor='red', markersize=10),
    Line2D([0], [0], marker='s', color='w', label='Negative IOD',
           markerfacecolor='blue', markersize=10),
    Line2D([0], [0], color='r', lw=4, alpha=0.5, label='High precip index'),
    Line2D([0], [0], color='gray', lw=4, alpha=0.5, label='Low precip index')
    ]
ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1.02, 0.5), fontsize=16,frameon=False)
ax.set_title("Interannual variation JJA rainfall", fontsize=20)
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-4.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
highyear_not_marked = [year for year in highyear if year not in marked_years]
lowyear_not_marked = [year for year in lowyear if year not in marked_years]

In [ ]:
high_positions = np.where(np.isin(years, highyear_not_marked))[0]
low_positions = np.where(np.isin(years, lowyear_not_marked))[0]
print("high_positions:", high_positions)
print("low_positions:", low_positions)

In [ ]:
print("high_ISM_years:",years[high_positions])
print("low_ISM_years:",years[low_positions])

In [ ]:
gph = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/gph_200hPa_1979-2024.npy")
uwnd = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/uwnd_200hPa_1979-2024.npy")
vwnd = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/vwnd_200hPa_1979-2024.npy")
u10 = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/u10_1979-2024.npy")
v10 = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/v10_1979-2024.npy")
t2m = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/t2m_1979-2024.npy")
slp = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/slp_1979-2024.npy")
lon = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/lon.npy")
lat = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/lat.npy")
ds_rws = xr.open_dataset("/public/home/songqh/project/ISM_impact_Antarctic/data/rws_output.nc")
rws =  ds_rws['s1']

In [ ]:
file0 = xr.open_dataset("/public/home/songqh/my_data/ice_cover/HadISST_ice.nc")
sic = file0['sic'].loc[file0.time.dt.year.isin(np.arange(1979,2025,1))
                        & file0.time.dt.month.isin([6,7,8])].groupby('time.year').mean('time')
sic_high = file0['sic'].loc[file0.time.dt.year.isin(np.arange(1979,2025,1)[high_positions]) 
            & file0.time.dt.month.isin([6,7,8])].groupby('time.year').mean('time')
sic_low = file0['sic'].loc[file0.time.dt.year.isin(np.arange(1979,2025,1)[low_positions]) 
            & file0.time.dt.month.isin([6,7,8])].groupby('time.year').mean('time')
lon0 = file0['longitude']
lat0 = file0['latitude']

In [ ]:
sic_high_low_diff, sic_high_low_p = scp.two_mean_test(sic_high,sic_low)
gph_high_low_diff, gph_high_low_p = scp.two_mean_test(gph[high_positions],gph[low_positions])
slp_high_low_diff, slp_high_low_p = scp.two_mean_test(slp[high_positions],slp[low_positions])
u10_high_low_diff, u10_high_low_p = scp.two_mean_test(u10[high_positions],u10[low_positions])
v10_high_low_diff, v10_high_low_p = scp.two_mean_test(v10[high_positions],v10[low_positions])
t2m_high_low_diff, t2m_high_low_p = scp.two_mean_test(t2m[high_positions],t2m[low_positions])
rws_high_low_diff, RWS_high_low_p = scp.two_mean_test(rws[high_positions],rws[low_positions])
u10_copy = np.where((u10_high_low_p > 0.05) & (v10_high_low_p > 0.05) , np.nan, u10_high_low_diff)
v10_copy = np.where((u10_high_low_p > 0.05) & (v10_high_low_p > 0.05) , np.nan, v10_high_low_diff)

In [ ]:
px = np.full((181,360),np.nan)
py = np.full((181,360),np.nan)
_,px,py= tnwaf(gph_high_low_diff,uwnd,vwnd,lon,lat,200)
px[:110],py[:110]=np.nan,np.nan
px[np.sqrt(px**2+py**2)<0.01]=np.nan
py[np.sqrt(px**2+py**2)<0.01]=np.nan

In [ ]:
def get_cmap_pr(n_colors: int, newcmap , method: int = None) -> list:
    index = list(range(1, n_colors + 1))
    color_list = [newcmap(i / n_colors) for i in index]
    if method == 1:
        color_list[0] = [1., 1., 1.]  
    elif method == 2:
        mid_index = len(color_list) // 2
        color_list[mid_index] = [1., 1., 1.]  
        color_list[mid_index - 1] = [1., 1., 1.]
    return color_list
new_CBR_coldhot = mpl.colors.ListedColormap(get_cmap_pr(18,cmaps.CBR_coldhot,method=2))
new_cmocean_balance_r = mpl.colors.ListedColormap(get_cmap_pr(20,cmaps.cmocean_balance_r,method=2))

In [ ]:
cycle_gph_high_low_diff,cycle_lon = add_cyclic_point(gph_high_low_diff, coord=lon)
cycle_rws_high_low_diff,cycle_lon = add_cyclic_point(rws_high_low_diff, coord=lon)
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(111,projection = ccrs.PlateCarree(central_longitude=150))
ax1.draw_map([-60,280,-90,30],latspec=30)
ax1.set_title("GPH&RWS&WAF@200hPa",fontsize=20,loc='center')
ax1.set_title("c",fontsize=24,weight='bold',loc='left')
cf1 = ax1.contourf(cycle_lon,lat,cycle_gph_high_low_diff/9.8, zorder=2,levels=np.arange(-200,220,20),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
rws_high_low_diff[160:,:] = np.nan
ax1.contour(cycle_lon,lat,cycle_rws_high_low_diff, zorder=2,levels=[2,4,6],colors='purple',transform=ccrs.PlateCarree())
ax1.contour(cycle_lon,lat,cycle_rws_high_low_diff, zorder=2,levels=[-6,-4,-2],colors='purple',transform=ccrs.PlateCarree())
ax1.contourf(lon,lat, gph_high_low_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
kk=8
q = ax1.quiver(lon[::kk],lat[::kk],px[::kk,::kk],py[::kk,::kk],alpha=0.7,color='g',scale=50, 
               width=0.003,headwidth=4, headlength=5, headaxislength=4, zorder=3,transform=ccrs.PlateCarree())
qk = ax1.quiverkey(q, 1.036, 0.99, U=2, label='2',labelpos='N',
        zorder=3, coordinates='axes',color='g',fontproperties={'size': 18},labelcolor='k')
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.5,pad=0.02)
cb.ax.set_colorbar(minor='off')
cb.set_ticks(np.arange(-160, 240, 80))
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-5c.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
cycle_slp_high_low_diff,cycle_lon = add_cyclic_point(slp_high_low_diff, coord=lon)
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(111,projection = ccrs.PlateCarree(central_longitude=150))
ax1.draw_map([-60,280,-90,30],latspec=30)
ax1.set_title("SLP&wind10m",fontsize=20,loc='center')
ax1.set_title("d",fontsize=24,weight='bold',loc='left')
cf1 = ax1.contourf(cycle_lon,lat,cycle_slp_high_low_diff, zorder=2,levels=np.arange(-900,1000,180),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
ax1.contourf(lon,lat, slp_high_low_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
q = ax1.quiver(lon[::6],lat[::6],u10_high_low_diff[::6,::6],v10_high_low_diff[::6,::6],zorder=3,
          scale=60,alpha=0.9,color='gray',transform=ccrs.PlateCarree())
q = ax1.quiver(lon[::6],lat[::6],u10_copy[::6,::6],v10_copy[::6,::6],zorder=3,color='k',
          scale=60,transform=ccrs.PlateCarree())
qk = ax1.quiverkey(q, 1.036, 0.99, U=3, label='3',lw=0.04,labelpos='N',zorder=3,
                    coordinates='axes',color='k',fontproperties={'size': 18},labelcolor='k')
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.5,pad=0.02)
cb.ax.set_colorbar(minor='off')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-5d.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
cycle_data,cycle_lon = add_cyclic_point(t2m_high_low_diff, coord=lon)
theta = np.linspace(0, 2*np.pi, 100)  
center, radius = [0.5, 0.5], 0.5  
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)  
fig = plt.figure(figsize=(5, 4))
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=120.0, central_latitude=-90.0, globe=None))
ax.set_extent([-180, 180, -50, -90], ccrs.PlateCarree())
gl = ax.gridlines(linestyle='--',zorder=2)
ax.set_boundary(circle, transform=ax.transAxes)  
ax.add_feature(cfeature.COASTLINE.with_scale('110m'),linewidth=1,zorder=3) 
u10_high_low_diff[170:182,:],v10_high_low_diff[170:182,:] = np.nan, np.nan
u10_copy[170:182,:],v10_copy[170:182,:] = np.nan, np.nan
kk = 6
q = ax.quiver(lon[::kk],lat[::kk],u10_high_low_diff[::kk,::kk],v10_high_low_diff[::kk,::kk],
           color='gray',alpha=0.9,zorder=3,scale=27,headwidth=5,width=0.004,transform=ccrs.PlateCarree())
q = ax.quiver(lon[::kk],lat[::kk],u10_copy[::kk,::kk],v10_copy[::kk,::kk],
              color='k',scale=27,headwidth=5,width=0.004,zorder=3,transform=ccrs.PlateCarree())
qk = ax.quiverkey(q, 1.07, 0.903, U=3, label='3',labelpos='N',
    zorder=3, coordinates='axes',color='k',fontproperties={'size': 18},
    labelcolor='k')
cf1 = ax.contourf(cycle_lon,lat,cycle_data, zorder=2,levels=np.arange(-5,6,1),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=cmaps.BlueWhiteOrangeRed)
ax.contourf(lon,lat, t2m_high_low_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.7,pad=0.035)
cb.ax.set_colorbar(minor='off')
ax.set_title("SAT&wind10m",fontsize=20,loc='center')
ax.set_title("a",fontsize=24,weight='bold',loc='left')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-5a.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
cycle_high_low_diff,cycle_lon = add_cyclic_point(sic_high_low_diff, coord=lon0)
leftlon, rightlon, lowerlat, upperlat = (-180,180,-50,-90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
fig = plt.figure(figsize=(5, 4))
ax1 = plt.axes(projection=ccrs.Orthographic(central_longitude=120.0, central_latitude=-90.0, globe=None))
ax1.gridlines(linestyle='--',zorder=2)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax1.set_boundary(circle, transform=ax1.transAxes)
ax1.add_feature(cfeature.LAND, facecolor='lightgray',zorder=3)
ax1.add_feature(cfeature.COASTLINE.with_scale('110m'),color='k',linewidth=1,zorder=3)
ax1.set_title("SIC",fontsize=20,loc='center')
ax1.set_title("b",fontsize=24,weight='bold',loc='left')
ax1.set_extent(img_extent, ccrs.PlateCarree())
cf1 = ax1.contourf(cycle_lon,lat0,cycle_high_low_diff, zorder=2,levels=np.linspace(-0.4,0.4,21),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_cmocean_balance_r)
ax1.contourf(lon0,lat0, sic_high_low_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none", transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.7,pad=0.035)
cb.ax.set_colorbar(minor='off')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-5b.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
file3 = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_vwnd_1979-2024.nc")
vwnd = file3['v'].loc[file3.time.dt.month.isin([6,7,8]),1000:100,60:-90,60:100].groupby('time.year').mean('time')
vwnd = vwnd[:,::-1,:,:]
vwnd_high = vwnd[high_positions]
vwnd_low = vwnd[low_positions]
lev = file3['level'].loc[1000:100].values
levv = lev[::-1]
latv = file3['latitude'].loc[60:-90].values

In [ ]:
va = vwnd.mean(('longitude'))
va_high = vwnd_high.mean(('longitude'))
va_low = vwnd_low.mean(('longitude'))
coslat = np.cos(lat*np.pi/180)
g = 9.8 
a = 6400000 
k1 = latv.shape[0]
msf = np.zeros([46,27,k1])
msf_high = np.zeros([len(high_positions),27,k1])
msf_low = np.zeros([len(low_positions),27,k1])
for t in range(46):
    for j in range(27):
        for k in range(k1): 
            f0 = np.trapz(va[t,0:j,k],levv[0:j],axis=0)
            msf[t,j,k] = 2*np.pi*a*f0*coslat[k]/g
for t in range(len(high_positions)):
    for j in range(27):
        for k in range(k1): 
            f0 = np.trapz(va_high[t,0:j,k],levv[0:j],axis=0)
            msf_high[t,j,k] = 2*np.pi*a*f0*coslat[k]/g
for t in range(len(low_positions)):
    for j in range(27):
        for k in range(k1): 
            f0 = np.trapz(va_low[t,0:j,k],levv[0:j],axis=0)
            msf_low[t,j,k] = 2*np.pi*a*f0*coslat[k]/g
msfu = msf/(10**9) 
msfu_high = msf_high/(10**9) 
msfu_low = msf_low/(10**9) 

In [ ]:
msfu_high_low_diff, msfu_high_low_p = scp.two_mean_test(msfu[high_positions],msfu[low_positions])

In [ ]:
fig = plt.figure(figsize=(16.45,4))
ax1 = plt.subplot(111)
ax1.set_detail()
ax1.set_ylabel('High (hPa)',fontsize=20)
ax1.set_yscale('symlog')
ax1.set_yticks([200,300,500,700,850,1000])
ax1.set_yticklabels(['200','300','500','700','850','1000'],fontsize=20)
ax1.set_ylim(1000,200) 
ax1.set_xlim(-90,30) 
ax1.axvline(10,c='g',lw=2.5)
ax1.axvline(28,c='g',lw=2.5)
ax1.set_xticks([-90,-75,-60,-45,-30,-15,0,15,30])
ax1.set_xticklabels([r'90$^\degree$S',r'75$^\degree$S',
        r'60$^\degree$S',r'45$^\degree$S', r'30$^\degree$S',
         r'15$^\degree$S', r'0$^\degree$',
         r'15$^\degree$N',  r'30$^\degree$N'] ,fontsize=20)
pos = fig.add_axes([0.911,0.175,0.008,0.65])
c1 = ax1.contourf(latv,levv,msfu_high_low_diff,
    levels=np.linspace(-2,2,11), 
    extend = 'both',zorder=0, cmap=cmaps.MPL_RdBu_r)
c2 = ax1.contour(latv,levv,msfu.mean(0),levels=np.linspace(-7,7,11), 
        extend = 'both',zorder=1, colors = 'k')
ax1.contourf(latv,levv, msfu_high_low_p, levels =[0,0.05,1],hatches=['..', None], 
                 zorder=3,colors="none")
cb = fig.colorbar(c1,orientation='vertical',format='%.2f',cax=pos)
cb.ax.set_colorbar(minor='off',fontsize=18)
ax1.set_title("Mass stream function",fontsize=20,loc='center')
ax1.set_title("e",fontsize=24,weight='bold',loc='left')
fig.savefig('/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-5e.jpeg',dpi=600, bbox_inches='tight')